## Setup

We define and implement the functions related to RSA cryptography.

### RSA Functions

In [1]:
function find_inv(x, modp)
    ~, ~, x_inv = gcdx(modp, x);  # NB x_inv is not guaranteed positive
    x_inv = mod(x_inv, modp);   # This ensures that x_inv is strictly positive
    return x_inv;
end;

function find_coprime(x, lowerbound = 1)
    for i::Int = lowerbound+1:typemax(Int)
        if gcd(i, x) == 1
            return i;
        end;
    end;
end;

function RSA_gen_keys(p1, p2)
    n = p1 * p2;
    t = (p1 - 1) * (p2 - 1);
    e = find_coprime(t, 10000);
    d = find_inv(e, t)
    public_key = (n, e);
    private_key = d;
    return public_key, private_key;
end;

function RSA_Encrypt(message, public_key)
    n, e = public_key;
    return powermod(message, e, n);
end;

function RSA_Decrypt(cipher, public_key, private_key)
    n, e = public_key;
    return powermod(cipher, private_key, n);
end;

### Key Generation

In [2]:
@show p1 = 70783;
@show p2 = 102059;

@show public_key, private_key = RSA_gen_keys(p1, p2);

p1 = 70783 = 70783
p2 = 102059 = 102059
(public_key, private_key) = RSA_gen_keys(p1, p2) = ((7224042197, 10001), 1381788029)


## Examples

### Arithmetic

We explore using RSA for homomorphic multiplication.

#### Basic Multiplication

Multiply two encrypted integers.

In [3]:
message = 2;

n, e = public_key;

ciphertext = RSA_Encrypt(message, public_key);

factor = RSA_Encrypt(2, public_key);

decoded = RSA_Decrypt(mod(ciphertext * factor, n), public_key, private_key);

@show decoded;

decoded = 4


#### Array Product

Here we explore an example of finding the product of elements in an array.

Imagine that Alice has some numbers which she wants to find the product of, but she is too lazy to do the calculations herself.

Instead, she uses her RSA public key to encrypt the elements of the array and sends it to Bob.

Bob, unaware of what he is working with, blindly multiplies the ciphertext together, and sends it back to Alice.

Alice finally uses her private key to retrieve the plaintext of the result. 

In [4]:
encrypt = x -> RSA_Encrypt(x, public_key);

multiply = (x, y) -> mod(Int128(x) * Int128(y), n);

code = map(encrypt, [2, 3, 4, 5]);

code_product = reduce(multiply, code);

product = RSA_Decrypt(code_product, public_key, private_key);

@show product;

product = 120


#### Find Square Root

Next we explore finding the the square root homomorphically.

Imagine Alice has a number, and she wants to find the square root of it but does not want to do the work herself.
Alice encrypts the number using her public key and sends it to Bob for him to find the EXACT square root of.

Bob then searches for a number whose square is exactly the ciphertext he recieved from Alice.
Because the encryption process in not order preserving, he cannot use the normal optimisation of searching from 2 to the square root of the number.

Instead Bob must do a search of complexity $\mathcal{O}(n)$, where $n$ is part of the public key, to find the sqaure root.
Sadly, this is comparable to the complexity of Bob doing a brute force search to find the plaintext corresponding to the ciphertext, so perhaps this is an impracticle use case of RSA for Homomorphic computing.

NOTE: This cell takes a LONG time to run.

In [ ]:
encrypt = x -> RSA_Encrypt(x, public_key);
multiply = (x, y) -> mod(Int128(x) * Int128(y), n)
power = (x, y) -> mod(reduce(multiply, ones(Int, y) * x), n);

message = 9;

@show code = encrypt(message)

_, _, limit = gcdx(code, 2)
limit = mod(limit, n)

code_root = 0;
for i = 2:limit
    if powermod(i, 2, n) == code
        code_root = i
        break;
    end    
end

@show code_root
root = RSA_Decrypt(code_root, public_key, private_key);

@show root;

### Logic


#### AND

The logical AND operation is analoguos to multiplication.


In [5]:
message = 1

n, e = public_key;

ciphertext = RSA_Encrypt(message, public_key);

@show ciphertext

state = RSA_Encrypt(0, public_key);

@show state

decoded = RSA_Decrypt(mod(ciphertext * state, n), public_key, private_key);

@show decoded;

ciphertext = 1
state = 0
decoded = 0


In [6]:
message = 1

n, e = public_key;

ciphertext = RSA_Encrypt(message, public_key);

@show ciphertext

state = RSA_Encrypt(1, public_key);

@show state

decoded = RSA_Decrypt(mod(ciphertext * state, n), public_key, private_key);

@show decoded;

ciphertext = 1
state = 1
decoded = 1


Of course, due to our basic implementation of RSA Encryption, encrypting a 1 or a 0 is pretty useless

Being mathematical identities, their properties makes any ciphertext very transparent and obvious to decode.

$\text{RSE_Encrypt}(0)=0$

$\text{RSE_Encrypt}(1)=1$

Instead, we could consider adding randomness to obviouscate it.

In this new representation, we represent 0 as any even number, and 1 with any odd number.

The behaviour of multiplying odd and even numbers mimics the truth table of an AND gate.

$\text{even} \cdot \text{even} = \text{even}, \quad 0\land0=0$

$\text{even} \cdot \text{odd} = \text{even}, \quad 0\land1=0$

$\text{odd} \cdot \text{odd} = \text{odd}, \quad 1\land1=1$

Bob calculates products, oblivious to their encrypted meaning (without significant work which is then not useful in quickly decoding future messages)

When Alice receives and decrypts the product, she checks if the product is odd to see if it is true.

In [7]:
using Random

message = 1
noise = rand(2:n-2)
randomisedMessage = noise + mod(noise, 2) + message;

ciphertext = RSA_Encrypt(randomisedMessage, public_key);
@show ciphertext

state = 0;
noise = rand(2:n-2);
randomisedState = noise + mod(noise, 2) + state;

cipherState = RSA_Encrypt(randomisedState, public_key);
@show cipherState

decoded = RSA_Decrypt(mod(ciphertext * cipherState, n), public_key, private_key);
@show decoded
@show mod(decoded, 2);

ciphertext = 599909037
cipherState = 3049898414
decoded = 719635182
mod(decoded, 2) = 0
